In [2]:
!wget -q https://git.io/J0fjL -O IAM_Words.zip
!unzip -qq IAM_Words.zip

!mkdir data
!mkdir data/words
!tar -xf IAM_Words/words.tgz -C data/words
!mv IAM_Words/words.txt data

In [ ]:
!head -20 data/words.txt

In [ ]:
!pip install --upgrade tensorflow

imports

In [5]:
# Instead of:
# from tensorflow.keras.layers.preprocessing import StringLookup

# Use:
from tensorflow.keras.layers import StringLookup # Import StringLookup directly from tf.keras.layers

from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

np.random.seed(42)
tf.random.set_seed(42)

dataset soliting

In [6]:
base_path = "data"
words_list = []

words = open(f"{base_path}/words.txt", "r").readlines()
for line in words:
  if line[0]=="#":
      continue
  if line.split(" ") [1] != "err":
    words_list.append(line)

len(words_list)

np.random.shuffle(words_list)

In [ ]:
print(words_list[0:10])

In [ ]:
from posixpath import split
split_idx = int(0.9 * len(words_list))
train_samples = words_list[:split_idx]
test_samples = words_list[split_idx:]

val_split_idx = int(0.5 * len(test_samples))
validation_samples = test_samples[:val_split_idx]
test_samples = test_samples[val_split_idx:]

assert len(words_list) == len(train_samples) + len(validation_samples) + len(test_samples)

print(f"Total training samples: {len(train_samples)}")
print(f"Total validation samples: {len(validation_samples)}")
print(f"Total test samples: {len(test_samples)}")

In [9]:
from sys import path
base_image_path = os.path.join(base_path, "words")

def get_image_paths_and_labels(samples):
    paths = []
    corrected_samples = []
    for (i, file_line) in enumerate(samples):
      line_split = file_line.strip()
      line_split = line_split.split(" ")
#Each line split will have this format for the corresponding image:
# part1/part1-part2/part1-part2-part3.png
      image_name = line_split[0]
      partI = image_name.split("-")[0]
      partII = image_name.split("-")[1]
      img_path = os.path.join(
          base_image_path, partI, partI + "-" + partII, image_name + ".png"
      )
      if os.path.getsize(img_path):
          paths.append(img_path)
          corrected_samples.append(file_line.split("\n")[0])

    return paths, corrected_samples

train_img_paths, train_labels = get_image_paths_and_labels(train_samples)
validation_img_paths, validation_labels = get_image_paths_and_labels(validation_samples)
test_img_paths, test_labels = get_image_paths_and_labels(test_samples)

In [ ]:
train_img_paths[0:10]

In [ ]:
train_labels[0:10]

In [ ]:
train_labels_cleaned = []
characters = set()
max_len = 0

for label in train_labels:
  label = label.split(" ")[-1].strip()
  for char in label:
    characters.add(char)

  max_len = max(max_len, len(label))
  train_labels_cleaned.append(label)

# Now print the results outside the loop
print(f"Maximum length: {max_len}")
print(f"Vocab size: {len(characters)}")

# Check some label samples
train_labels_cleaned[:10]


then we prepare ground-truth labels

now we clean the validation and test lables as well

In [13]:
def clean_labels(labels):
    cleaned_labels = []
    for label in labels:
        label = label.split(" ")[-1].strip()
        cleaned_labels.append(label)
    return cleaned_labels

validation_labels_cleaned = clean_labels(validation_labels)
test_labels_cleaned = clean_labels(test_labels)

building the character vocabulary

In [14]:
AUTOTUNE = tf.data.AUTOTUNE

# Mapping characters to integers
char_to_num = StringLookup(
    vocabulary=list(characters), mask_token=None
)

# mapping integer to character
num_to_char = StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

Resizing image without distortion

In [15]:
def distortion_free_resize(image, img_size):
    w, h = img_size
    image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

    # check the amount of padding need to be done
    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    # only works for even dimensions
    if pad_height % 2 != 0:
        height = pad_height // 2
        pad_height_top = height + 1
        pad_height_bottom = height
    else:
        pad_height_top = pad_height_bottom = pad_height //2

    if pad_width % 2 != 0:
        width  = pad_width // 2
        pad_width_left = width + 1
        pad_width_right = width
    else:
        pad_width_left  = pad_width_right = pad_width // 2

    image = tf.pad(
        image,
        paddings=[
          [pad_height_top, pad_height_bottom],
          [pad_width_left, pad_width_right],
          [0, 0],
        ],
  )

    image = tf.transpose(image, perm=[1, 0, 2])
    image = tf.image.flip_left_right(image)
    return image

In [16]:
batch_size = 64
padding_token = 99
image_width = 128
image_height = 32

def preprocess_image(image_path, img_size=(image_width, image_height)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = distortion_free_resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def vectorize_label(label):
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    length = tf.shape(label)[0]
    pad_amount = max_len - length
    label = tf.pad(label, paddings=[[0, pad_amount]], constant_values=padding_token)
    return label

def process_images_labels(image_path, label):
    image = preprocess_image(image_path)
    label = vectorize_label(label)
    return {"image": image, "label": label}

def prepare_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels)).map(
        process_images_labels, num_parallel_calls=AUTOTUNE
    )
    return dataset.batch(batch_size).cache().prefetch(AUTOTUNE)

prepare tf data Dataset object

In [17]:
train_ds = prepare_dataset(train_img_paths, train_labels_cleaned)
validation_ds = prepare_dataset(validation_img_paths, validation_labels_cleaned)
test_ds = prepare_dataset(test_img_paths, test_labels_cleaned)

visualize few sample

In [ ]:
for data in train_ds.take(1):
    images, labels = data["image"], data["label"]
    _, ax = plt.subplots(4, 4, figsize=(15, 8))
    for i in range(16):
        img = images[i]
        img = tf.image.flip_left_right(img)
        img = tf.transpose(img, perm=[1, 0, 2])
        img = (img * 255.0).numpy().clip(0, 255).astype(np.uint8)
        img = img[:, :, 0]
        label = labels[i]
        indices = tf.gather(label, tf.where(tf.math.not_equal(label, padding_token)))
        label = tf.strings.reduce_join(num_to_char(indices))
        label = label.numpy().decode("utf-8")
        ax[i//4, i % 4].imshow(img, cmap="gray")
        ax[i//4, i % 4].set_title(label)
        ax[i//4, i % 4].axis("off")

plt.show()

Model

In [ ]:
class CTCLayer(keras.layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

def build_model():
    input_img = keras.Input(shape=(image_width, image_height, 1), name="image")
    labels = keras.layers.Input(name="label", shape=(None,))
    x = keras.layers.Conv2D(
        32,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(input_img)
    x = keras.layers.MaxPooling2D((2, 2), name="pool1")(x)


    x = keras.layers.Conv2D(
        64,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv2",
    )(x)
    x = keras.layers.MaxPooling2D((2, 2), name="pool2")(x)


    # parsing to the output
    new_shape = ((image_width // 4), (image_height // 4) * 64)
    x = keras.layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = keras.layers.Dense(64, activation="relu", name="dense1")(x)
    x = keras.layers.Dropout(0.2)(x)

    # RNNs
    x = keras.layers.Bidirectional(
        keras.layers.LSTM(128, return_sequences=True, dropout=0.25)
    )(x)
    x = keras.layers.Bidirectional(
        keras.layers.LSTM(64, return_sequences=True, dropout=0.25)
    )(x)
    x = keras.layers.Dense(
        len(char_to_num.get_vocabulary()) + 1, activation="softmax", name="dense2"
    )(x)

    # add CTC layer for claculating Ctc loss at each step
    output = CTCLayer(name="ctc_loss")(labels, x)

    # define the model
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="handwriting_recognizer"
    )

    # define the model
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt)
    return model

model = build_model()
model.summary()

Evaluation matric

In [20]:
validation_imqges = []
validation_labels = []

for batch in validation_ds:
    validation_imqges.append(batch["image"])
    validation_labels.append(batch["label"])

Now we create a call back for monitoring the editdistance

In [21]:
def claculate_edit_distance(labels, predictions):
    saprse_lables = tf.cast(tf.sparse.from_dense(labels), dtype=tf.int64)

    # make pridiction and conver them into sparse
    input_len = np.ones(predictions.shape[0]) * predictions.shape[1]
    predictions_decoded = keras.backend.ctc_decode(
        predictions, input_length=input_len, greedy=True)[0][0][:,:max_len]
    sparse_predictions = tf.cast(tf.sparse.from_dense(predictions_decoded), dtype=tf.int64)

    # compute the edit distance
    edit_distances = tf.edit_distance(sparse_predictions, saprse_lables, normalize=False)
    return tf.reduce_mean(edit_distances)


class EditDistanceCallback(keras.callbacks.Callback):
    def __init__(self, pred_model):
        super().__init__()
        self.prediction_model = pred_model

    def on_epoch_end(self, epoch, logs=None):
        edit_distances = []
        for i in range(len(validation_labels)):
            labels = validation_labels[i]
            predictions = self.prediction_model.predict(validation_imqges[i])
            edit_distances.append(claculate_edit_distance(labels, predictions).numpy())

        print(f"Mean edit distance for epoch {epoch + 1}: {np.mean(edit_distances):.4f}")

Tranning

In [ ]:
# Step 1: Install Required Libraries
!pip install easyocr
!pip install opencv-python-headless

# Step 2: Import Libraries
import easyocr
import cv2
from google.colab import files

# Step 3: Preprocess Image to Improve OCR Accuracy
def preprocess_image(image_path):
    """
    Preprocess the image by converting it to grayscale and applying binary thresholding.
    """
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
        if image is None:
            raise ValueError(f"Could not load image: {image_path}")
        _, processed_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)  # Apply binary thresholding
        cv2.imwrite(image_path, processed_image)  # Overwrite the original image with the processed version
        return image_path
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")
        return None

# Step 4: Extract Text from Images and Save to File
def save_all_text(images, output_file):
    """
    Extract text from multiple images using EasyOCR and save all extracted text to a single file.
    """
    reader = easyocr.Reader(['en'])  # Initialize EasyOCR
    extracted_texts = []

    with open(output_file, 'w') as file:
        for image in images:
            print(f"Processing {image}...")
            try:
                # Preprocess the image
                preprocessed_image = preprocess_image(image)
                if preprocessed_image is None:
                    continue

                # Extract text
                text = reader.readtext(preprocessed_image, detail=0)
                if text:
                    extracted_texts.append((image, text))
                    file.write(f"Text from {image}:\n")
                    file.write('\n'.join(text) + '\n\n')
                    print(f"Extracted text: {text}")
                else:
                    print(f"No text detected in {image}.")
                    file.write(f"Text from {image}:\nNo text detected.\n\n")
            except Exception as e:
                print(f"Error processing {image}: {e}")
    print(f"All extracted text has been saved to {output_file}")
    return extracted_texts

# Step 5: Upload Images to Colab
print("Please upload your images (e.g., PNG, JPG):")
uploaded = files.upload()

# Step 6: Process Uploaded Images
image_paths = list(uploaded.keys())  # Get the list of uploaded image filenames
output_file = "all_extracted_text.txt"  # Output file for extracted text

extracted_texts = save_all_text(image_paths, output_file)

# Step 7: Download the Output File
print("Download the extracted text file:")
files.download(output_file)

# Debugging: Print extracted text for review
if extracted_texts:
    for image, text in extracted_texts:
        print(f"\nText from {image}:\n{' '.join(text)}")
else:
    print("No text was extracted. Check image quality or preprocessing.")
